In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# External imports
import ipywidgets as ipw

from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import SmilesWidget
from aiidalab_widgets_base import BasicStructureEditor
from aiidalab_widgets_base import WizardAppWidget

In [ ]:
from aiidalab_ispg import ConformerSmilesWidget
from aiidalab_ispg import TrajectoryDataViewer
from aiidalab_ispg.widgets import TrajectoryManagerWidget

from aiidalab_ispg.qeapp.process import WorkChainSelector
from aiidalab_ispg import StructureSelectionStep
from aiidalab_ispg import SubmitAtmospecAppWorkChainStep
from aiidalab_ispg import ViewAtmospecAppWorkChainStatusAndResultsStep
from aiidalab_ispg import ViewSpectrumStep

structure_manager_widget = TrajectoryManagerWidget(
    importers=[
        ConformerSmilesWidget(title='SMILES'),
        StructureUploadWidget(title="Upload file"),
        StructureBrowserWidget(title="AiiDA database"),
    ],
    node_class='TrajectoryData',
    viewer=TrajectoryDataViewer(),
)

structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True

submit_orca_work_chain_step = SubmitAtmospecAppWorkChainStep()
submit_orca_work_chain_step.auto_advance = True

view_orca_work_chain_status_and_results_step = ViewAtmospecAppWorkChainStatusAndResultsStep()
view_orca_work_chain_status_and_results_step.auto_advance = True

view_spectrum_step = ViewSpectrumStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_orca_work_chain_step, 'input_structure'))
ipw.dlink((submit_orca_work_chain_step, 'process'), (view_orca_work_chain_status_and_results_step, 'process'))
ipw.dlink((view_orca_work_chain_status_and_results_step, 'process'), (view_spectrum_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit workflow', submit_orca_work_chain_step),
        ('Status & Detailed outputs', view_orca_work_chain_status_and_results_step),
        ('UV/VIS Spectrum', view_spectrum_step),
    ])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))
def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                structure_manager_widget.structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                submit_orca_work_chain_step.process = process

work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_orca_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)


app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

display(app_with_work_chain_selector)